# Cài đặt ứng dụng đơn giản của cây đỏ đen là từ điển EN-VN

In [ ]:
import sys

# 1. Cấu trúc Node
class Node:
    def __init__(self, key, value, color='RED'):
        self.key = key
        self.value = value
        self.color = color 
        self.parent = None
        self.left = None
        self.right = None

# 2. Cấu trúc Cây Đỏ-Đen
class RedBlackTree:
    def __init__(self):
        # TNULL là node lá ảo, luôn là màu Đen
        self.TNULL = Node(None, None, 'BLACK')
        self.TNULL.left = None
        self.TNULL.right = None
        self.root = self.TNULL

    # --- CÁC HÀM BỔ TRỢ CƠ BẢN ---
    def search_tree(self, k):
        return self._search_tree_helper(self.root, k)

    def _search_tree_helper(self, node, key):
        if node == self.TNULL or key == node.key:
            return node
        if key < node.key:
            return self._search_tree_helper(node.left, key)
        return self._search_tree_helper(node.right, key)

    def minimum(self, node):
        while node.left != self.TNULL:
            node = node.left
        return node

    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x
        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.TNULL:
            y.right.parent = x
        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y

    # Thay thế cây con u bằng cây con v
    def transplant(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    # --- PHẦN INSERT (THÊM) ---
    def insert(self, key, value):
        node = Node(key, value)
        node.parent = None
        node.left = self.TNULL
        node.right = self.TNULL
        node.color = 'RED' # Node mới luôn đỏ

        y = None
        x = self.root

        while x != self.TNULL:
            y = x
            if node.key < x.key:
                x = x.left
            elif node.key > x.key:
                x = x.right
            else:
                # Nếu từ đã tồn tại, cập nhật nghĩa
                x.value = value
                return

        node.parent = y
        if y is None:
            self.root = node
        elif node.key < y.key:
            y.left = node
        else:
            y.right = node

        if node.parent is None:
            node.color = 'BLACK'
            return

        if node.parent.parent is None:
            return

        self.fix_insert(node)

    def fix_insert(self, k):
        while k.parent.color == 'RED':
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left
                if u.color == 'RED':
                    u.color = 'BLACK'
                    k.parent.color = 'BLACK'
                    k.parent.parent.color = 'RED'
                    k = k.parent.parent
                else:
                    if k == k.parent.left:
                        k = k.parent
                        self.right_rotate(k)
                    k.parent.color = 'BLACK'
                    k.parent.parent.color = 'RED'
                    self.left_rotate(k.parent.parent)
            else:
                u = k.parent.parent.right
                if u.color == 'RED':
                    u.color = 'BLACK'
                    k.parent.color = 'BLACK'
                    k.parent.parent.color = 'RED'
                    k = k.parent.parent
                else:
                    if k == k.parent.right:
                        k = k.parent
                        self.left_rotate(k)
                    k.parent.color = 'BLACK'
                    k.parent.parent.color = 'RED'
                    self.right_rotate(k.parent.parent)
            if k == self.root:
                break
        self.root.color = 'BLACK'

    # --- PHẦN DELETE (XÓA) ---
    def delete_node(self, key):
        z = self.TNULL
        curr = self.root
        while curr != self.TNULL:
            if curr.key == key:
                z = curr
            if curr.key <= key:
                curr = curr.right
            else:
                curr = curr.left

        if z == self.TNULL:
            print(f"Không tìm thấy từ '{key}' để xóa.")
            return

        y = z
        y_original_color = y.color
        x = self.TNULL

        if z.left == self.TNULL:
            x = z.right
            self.transplant(z, z.right)
        elif z.right == self.TNULL:
            x = z.left
            self.transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                x.parent = y
            else:
                self.transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color

        if y_original_color == 'BLACK':
            self.fix_delete(x)
        
        print(f"Đã xóa: '{key}'")

    def fix_delete(self, x):
        while x != self.root and x.color == 'BLACK':
            if x == x.parent.left:
                s = x.parent.right # Node anh em (Sibling)
                if s.color == 'RED':
                    s.color = 'BLACK'
                    x.parent.color = 'RED'
                    self.left_rotate(x.parent)
                    s = x.parent.right

                if s.left.color == 'BLACK' and s.right.color == 'BLACK':
                    s.color = 'RED'
                    x = x.parent
                else:
                    if s.right.color == 'BLACK':
                        s.left.color = 'BLACK'
                        s.color = 'RED'
                        self.right_rotate(s)
                        s = x.parent.right
                    
                    s.color = x.parent.color
                    x.parent.color = 'BLACK'
                    s.right.color = 'BLACK'
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                s = x.parent.left
                if s.color == 'RED':
                    s.color = 'BLACK'
                    x.parent.color = 'RED'
                    self.right_rotate(x.parent)
                    s = x.parent.left

                if s.right.color == 'BLACK' and s.left.color == 'BLACK':
                    s.color = 'RED'
                    x = x.parent
                else:
                    if s.left.color == 'BLACK':
                        s.right.color = 'BLACK'
                        s.color = 'RED'
                        self.left_rotate(s)
                        s = x.parent.left

                    s.color = x.parent.color
                    x.parent.color = 'BLACK'
                    s.left.color = 'BLACK'
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 'BLACK'

# 3. Lớp Ứng Dụng: Từ điển
class EnglishVietnameseDictionary:
    def __init__(self):
        self.bst = RedBlackTree()

    def add_word(self, en_word, vn_meaning):
        self.bst.insert(en_word.lower(), vn_meaning)
        print(f"Thêm: '{en_word}' -> '{vn_meaning}'")

    def remove_word(self, en_word):
        self.bst.delete_node(en_word.lower())

    def lookup(self, en_word):
        node = self.bst.search_tree(en_word.lower())
        if node != self.bst.TNULL:
            return f"[Tra cứu] {en_word}: {node.value}"
        else:
            return f"[Tra cứu] '{en_word}' chưa có trong từ điển."

    # Hàm in cây để kiểm tra cấu trúc (Debug)
    def print_structure(self):
        print("\n--- Cấu trúc cây hiện tại ---")
        self._print_helper(self.bst.root, "", True)
        print("-----------------------------\n")

    def _print_helper(self, node, indent, last):
        if node != self.bst.TNULL:
            sys.stdout.write(indent)
            if last:
                sys.stdout.write("R----")
                indent += "     "
            else:
                sys.stdout.write("L----")
                indent += "|    "
            
            s_color = "ĐỎ" if node.color == "RED" else "ĐEN"
            print(f"{node.key} ({s_color})")
            self._print_helper(node.left, indent, False)
            self._print_helper(node.right, indent, True)

# --- CHẠY THỬ CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    my_dict = EnglishVietnameseDictionary()

    # 1. Thêm dữ liệu
    my_dict.add_word("apple", "Quả táo")
    my_dict.add_word("banana", "Quả chuối")
    my_dict.add_word("cherry", "Quả anh đào")
    my_dict.add_word("date", "Quả chà là")
    my_dict.add_word("elderberry", "Quả cơm cháy")
    
    # In cấu trúc để xem màu Đỏ/Đen được phân bố thế nào
    my_dict.print_structure()

    # 2. Tra cứu
    print(my_dict.lookup("Banana"))
    print(my_dict.lookup("Fig"))

    # 3. Xóa dữ liệu (Thử thách cho cây)
    print("\n--- Bắt đầu xóa ---")
    my_dict.remove_word("banana") # Xóa một node giữa cây
    my_dict.remove_word("apple")  # Xóa node gốc hoặc node biên
    
    # 4. Kiểm tra lại sau khi xóa
    print(my_dict.lookup("banana"))
    my_dict.print_structure()

Thêm: 'apple' -> 'Quả táo'
Thêm: 'banana' -> 'Quả chuối'
Thêm: 'cherry' -> 'Quả anh đào'
Thêm: 'date' -> 'Quả chà là'
Thêm: 'elderberry' -> 'Quả cơm cháy'

--- Cấu trúc cây hiện tại ---
R----banana (ĐEN)
     L----apple (ĐEN)
     R----date (ĐEN)
          L----cherry (ĐỎ)
          R----elderberry (ĐỎ)
-----------------------------

[Tra cứu] Banana: Quả chuối
[Tra cứu] 'Fig' chưa có trong từ điển.

--- Bắt đầu xóa ---
Đã xóa: 'banana'
Đã xóa: 'apple'
[Tra cứu] 'banana' chưa có trong từ điển.

--- Cấu trúc cây hiện tại ---
R----date (ĐEN)
     L----cherry (ĐEN)
     R----elderberry (ĐEN)
-----------------------------

